In [1]:
import re
import json

Открываю вручную разбитый на предложения кусок текста:

In [2]:
f = open('part.json', 'r', encoding = 'utf-8')
json_string = f.read()
f.close()
gold = json.loads(json_string)
gold

['В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, в какой ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян,\xa0— словом, все те, которых называют господами средней руки.',
 'В бричке сидел господин, не красавец, но и не дурной наружности, ни слишком толст, ни слишком тонок; нельзя сказать, чтобы стар, однако ж и не так, чтобы слишком молод.',
 'Въезд его не произвел в городе совершенно никакого шума и не был сопровожден ничем особенным; только два русские мужика, стоявшие у дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, более к экипажу, чем к сидевшему в нем.',
 '«Вишь ты,\xa0— сказал один другому,\xa0— вон какое колесо! что ты думаешь, доедет то колесо, если б случилось, в Москву или не доедет?» — «Доедет»,\xa0— отвечал другой.',
 '«А в Казань-то, я думаю, не доедет?» — «В Казань не доедет»,\xa0— отвечал другой.',
 'Этим разговор и кончился.',
 'Да ещ

In [3]:
f = open('part.txt', 'r', encoding = 'utf-8')
text = f.read()
f.close()

In [4]:
regex = '(?<=[\.\?!])\s(?=[A-ZА-ЯЁ])'

Разбиваю кусок текста на предложения с помощью регулярного выражения:

In [5]:
my_split = re.split(regex, text)
my_split

['В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, в какой ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян,\xa0— словом, все те, которых называют господами средней руки.',
 'В бричке сидел господин, не красавец, но и не дурной наружности, ни слишком толст, ни слишком тонок; нельзя сказать, чтобы стар, однако ж и не так, чтобы слишком молод.',
 'Въезд его не произвел в городе совершенно никакого шума и не был сопровожден ничем особенным; только два русские мужика, стоявшие у дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, более к экипажу, чем к сидевшему в нем. «Вишь ты,\xa0— сказал один другому,\xa0— вон какое колесо! что ты думаешь, доедет то колесо, если б случилось, в Москву или не доедет?» — «Доедет»,\xa0— отвечал другой. «А в Казань-то, я думаю, не доедет?» — «В Казань не доедет»,\xa0— отвечал другой.',
 'Этим разговор и кончился.',
 'Да еще, когда

Оцениваю качество разбиения, считая пересечения множеств:

In [6]:
len(set(gold) & set(my_split)) / len(set(gold) | set(my_split))

0.6909090909090909

In [7]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

Обучаю PunktSentenceTokenizer на всем тексте:

In [8]:
f = open('mertvye_dushi.txt', 'r', encoding = 'utf-8')
full_text = f.read()
f.close()

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(full_text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

Разбиваю кусок текста с помощью модели:

In [9]:
new_split = tokenizer.tokenize(text)

In [10]:
new_split

['В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, в какой ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян,\xa0— словом, все те, которых называют господами средней руки.',
 'В бричке сидел господин, не красавец, но и не дурной наружности, ни слишком толст, ни слишком тонок; нельзя сказать, чтобы стар, однако ж и не так, чтобы слишком молод.',
 'Въезд его не произвел в городе совершенно никакого шума и не был сопровожден ничем особенным; только два русские мужика, стоявшие у дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, более к экипажу, чем к сидевшему в нем.',
 '«Вишь ты,\xa0— сказал один другому,\xa0— вон какое колесо!',
 'что ты думаешь, доедет то колесо, если б случилось, в Москву или не доедет?» — «Доедет»,\xa0— отвечал другой.',
 '«А в Казань-то, я думаю, не доедет?» — «В Казань не доедет»,\xa0— отвечал другой.',
 'Этим разговор и кончился.',
 'Д

Оцениваю качество:

In [11]:
len(set(gold) & set(new_split)) / len(set(gold) | set(new_split))

0.94

Модель разбивает одно предложение посреди реплики где идет восклицательный знак, а после него маленькая буква. Я не разделяла это на отдельные предложения, когда разбивала вручную, но не уверена как было бы правильнее, и как вообще разбивать на предложения в местах с чьими-то репликами.